In [ ]:
import sys
import os
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import pandas as pd
import xarray as xr
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
from lesview import *
from gotmtool import *
from sbl_bbl import *

In [ ]:
g = 9.81
H = 30
u10 = 8
N2 = 1.962e-4
bstar = N2 * H
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
lat = 45.
alphaT = 2.0e-4
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)
amplitude = 1.0
wavelength = 60
wavenumber = 2.*np.pi/wavelength
frequency = np.sqrt(g*wavenumber*np.tanh(wavenumber*H))
us0 = amplitude**2*wavenumber
la = np.sqrt(ustar/us0) 
print('La = {:6.3f}'.format(la))
Ti = inertial_period(lat)
print('Ti = {:6.3f}'.format(Ti))

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2'
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
turbmethod = 'SMCLT-H15'
runs = {'r1': '', 'r2': '_rf'}
ds_gotm = {}
ds_ocgn = {}
for rkey in runs.keys():
    gotm_dir = os.path.join(os.path.pardir, 'gotm', 'run', '{:s}{:s}'.format(casename, runs[rkey]))
    gotm_sim = Simulation(path=os.path.join(gotm_dir, turbmethod))
    ds_gotm[rkey] = gotm_sim.load_data()
    ocgn_dir = os.path.join(os.path.pardir, 'tests', '{:s}{:s}'.format(casename, runs[rkey]))
    filepath = os.path.join(ocgn_dir, 'averages.jld2')
    ds_ocgn[rkey] = OceananigansDataProfile(filepath=filepath).dataset

In [ ]:
tslice_gotm = {}
tslice_ocgn = {}
for rkey in runs.keys():
    print('---- GOTM {:s} ----'.format(rkey))
    N2_gotm = nondim_da(ds_gotm[rkey].data_vars['temp'][:,:,0,0].differentiate(coord='z')*alphaT*g/N2, H=H, Tf=Ti)
    tslice_gotm[rkey] = get_tslice(N2_gotm, Ti)
    print('---- Oceananigans {:s} ----'.format(rkey))
    N2_ocgn = nondim_da(ds_ocgn[rkey].data_vars['b'].differentiate(coord='z')/N2, H=H, Tf=Ti)
    tslice_ocgn[rkey] = get_tslice(N2_ocgn, Ti)

In [ ]:
def get_das_gotm(ds_gotm):
    das_gotm = dict(
            b  = (ds_gotm.data_vars['temp']-17)*alphaT*g/bstar,
            u  = (ds_gotm.data_vars['u']+ds_gotm.data_vars['us'])/ustar,
            v  = (ds_gotm.data_vars['v']+ds_gotm.data_vars['vs'])/ustar,
            wb = get_flux(ds_gotm.data_vars['temp'].squeeze(),
                          ds_gotm.data_vars['nuh'].squeeze(),
                          ds_gotm.data_vars['gamh'].squeeze())*alphaT*g/ustar/bstar*10**3,
            wu = (get_flux(ds_gotm.data_vars['u'].squeeze(),
                          ds_gotm.data_vars['num'].squeeze(),
                          ds_gotm.data_vars['gamu'].squeeze())
                  -ds_gotm.data_vars['nucl'].squeeze()*ds_gotm.data_vars['dusdz'].squeeze())/ustar**2,
            wv = (get_flux(ds_gotm.data_vars['v'].squeeze(),
                          ds_gotm.data_vars['num'].squeeze(),
                          ds_gotm.data_vars['gamv'].squeeze())
                  -ds_gotm.data_vars['nucl'].squeeze()*ds_gotm.data_vars['dvsdz'].squeeze())/ustar**2,
        )
    return das_gotm

def get_das_ocgn(ds_ocgn):
    das_ocgn = dict(
            b  = ds_ocgn.data_vars['b']/bstar,
            u  = ds_ocgn.data_vars['u']/ustar,
            v  = ds_ocgn.data_vars['v']/ustar,
            wb = (ds_ocgn.data_vars['wb']+ds_ocgn.data_vars['wbsb'])/ustar/bstar*10**3,
            wu = (ds_ocgn.data_vars['wu']+ds_ocgn.data_vars['wusb'])/ustar**2,
            wv = (ds_ocgn.data_vars['wv']+ds_ocgn.data_vars['wvsb'])/ustar**2,
        )
    return das_ocgn

In [ ]:
labels = dict(
    b  = '$\overline{b}/b_*$',
    u  = '$\overline{u}/u_*$',
    v  = '$\overline{v}/u_*$',
    wb = '$10^3\overline{w^\prime b^\prime}/u_*b_*$',
    wu = '$\overline{w^\prime u^\prime}/u_*^2$',
    wv = '$\overline{w^\prime v^\prime}/u_*^2$',
)
loc = dict(
    b  = [0,0],
    u  = [1,0],
    v  = [2,0],
    wb = [0,1],
    wu = [1,1],
    wv = [2,1],
)
xylabel = dict(
    b  = [True, True],
    u  = [True, False],
    v  = [True, False],
    wb = [True, True],
    wu = [True, False],
    wv = [True, False],
)
abc = dict(
    b  = 'ag',
    u  = 'bh',
    v  = 'ci',
    wb = 'dj',
    wu = 'ek',
    wv = 'fl',
)
if 'rf' in casename:
    abc_u = 'right'
else:
    abc_u = 'left'
abc_loc = dict(
    b  = ['left',  'left'],
    u  = ['left',  'right'],
    v  = ['right', 'right'],
    wb = ['left',  'left'],
    wu = ['right', 'right'],
    wv = ['left',  'left'],
)
abc_x = {'left': 0.1, 'right': 0.9}
tags = ['Aligned', 'Opposite']

fig, axarr = plt.subplots(3, 4, sharey='row')
fig.set_size_inches([9, 8])

for k, rkey in enumerate(runs.keys()):
    das_gotm = nondim_das(get_das_gotm(ds_gotm[rkey]), H=H, Tf=Ti)
    das_ocgn = nondim_das(get_das_ocgn(ds_ocgn[rkey]), H=H, Tf=Ti)
    tslice1_gotm, tslice2_gotm = tslice_gotm[rkey]
    tslice1_ocgn, tslice2_ocgn = tslice_ocgn[rkey]
    for i, var in enumerate(das_ocgn.keys()):
        al = loc[var]
        ax = axarr[al[0], al[1]+k*2]
        da1 = das_ocgn[var].sel(time=tslice1_ocgn).mean(dim='time')
        l1, = da1.plot(ax=ax, y=da1.dims[0], linestyle='--', color='k')
        da2 = das_ocgn[var].sel(time=tslice2_ocgn).mean(dim='time')
        l2, = da2.plot(ax=ax, y=da2.dims[0], linestyle='-', color='k')
        da3 = das_gotm[var].sel(time=tslice1_gotm).mean(dim='time')
        l3, = da3.plot(ax=ax, y=da3.dims[0], linestyle='--', color='tab:blue')
        da4 = das_gotm[var].sel(time=tslice2_gotm).mean(dim='time')
        l4, = da4.plot(ax=ax, y=da4.dims[0], linestyle='-', color='tab:blue')
        ax.text(abc_x[abc_loc[var][k]], 0.9, '({:s})'.format(abc[var][k]), transform=ax.transAxes, va='top', ha=abc_loc[var][k])
        ax.set_title('')
        ax.set_xlabel(labels[var])
        ax.set_ylabel('')
for i in np.arange(3):
    ax = axarr[i,0]
    ax.set_ylabel('$z/H$')
    if i == 0:
        ax.legend([l1, l2], ['T1', 'T2'], loc='center left', fontsize=9)
    elif i == 1:
        ax.legend([l2, l4], ['LES', turbmethod[6:]], loc='center left', fontsize=9)
for i in np.arange(2):
    ax = axarr[0,i*2]
    ax.text(1.1, 1.05, tags[i], transform=ax.transAxes, va='bottom', ha='center')
        
plt.subplots_adjust(top=0.95, bottom=0.08, left=0.08, right=0.97, hspace=0.35, wspace=0.15)
figname = os.path.join(figpath, 'profiles-v2-{:s}'.format(turbmethod))
fig.savefig(figname, dpi = 300, facecolor='w')